In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")

In [ ]:
df.columns

In [ ]:
df.dtypes

# Variables survey
- `id`: unique id (not use)
- `bin_0,1,2`: 0 or 1
- `ord_0`: 1,2,3
- `day`: means day of week

In [ ]:
for i in df.columns:
    if df[i].dtype != "object":
        print(df[i].value_counts())

- `bin_3,4`: mean true(T) or false(F), Yes(Y) or No(N)
- `non_5,6,7,8,9`: Too many categories (maybe hex value (36bit), we can convert continous value)

In [ ]:
for i in df.columns:
    if df[i].dtype == "object":
        print(df[i].value_counts())

In [ ]:
for i in df.columns:
    if df[i].dtype == "object":
        print(i+": "+str(df[i].nunique()))

# category vs target
- ord3,4: the latter letter has the higher mean value

In [ ]:
for i in df.columns:
    print(df.groupby(i).agg({"target":"mean"}))

In [ ]:
for i in [1,2,3,4,5,6,7]:
    print(np.cos(2*np.pi/6 * (i-1)))

In [ ]:
df["day"] = np.cos((2*np.pi/6 * (df.day-1)))

In [ ]:
df.groupby("day").agg({"target":"mean"})

In [ ]:
df.day.unique()

In [ ]:
df["ord_5_1"] = df.ord_5.apply(lambda x: x[0])
df["ord_5_2"] = df.ord_5.apply(lambda x: x[1])

In [ ]:
df.drop("ord_5",axis=1,inplace=True)

In [ ]:
df.groupby("ord_5_1").agg({"target":"mean"})

In [ ]:
df.groupby("ord_5_2").agg({"target":"mean"})

In [ ]:
df["bin_3"] = df.bin_3.replace({"F":0,"T":1})

In [ ]:
df["bin_4"] = df.bin_4.replace({"N":0,"Y":1})

In [ ]:
for i in ["nom_%d"%i for i in [5,6,7,8,9]]:
    df[i] = df[i].apply(lambda x:int(x,16))

- `nom_5,6,7,8,9`: continuos values don't have information...(use target encoding with smoothing)

In [ ]:
for i in [5,6,7,8,9]:
    value, bins = pd.qcut(df["nom_%d"%(i)],50,retbins=True)
    bins = ["<"+str(int(i)) for i in bins]
    plt.plot(bins[1:],pd.concat([df.target,value],axis=1).groupby("nom_%d"%(i)).agg({"target":"mean"}),marker="o")
    plt.title("nom_%d"%(i))
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
df.nom_5.describe()

In [ ]:
#for i in ["nom_%d"%i for i in [5,6,7,8,9]]:
#    for j in range(3):
#        df["%s_%d"%(i,j)] = df[i].apply(lambda x:int(x[j*3:(j+1)*3],16))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#for i in [5,6,7,8,9]:
#    for j in [0,1,2]:
#        value, bins = pd.qcut(df["nom_%d_%d"%(i,j)],10,retbins=True)
#        bins = ["<"+str(int(i)) for i in bins]
#        plt.plot(bins[1:],pd.concat([df.target,value],axis=1).groupby("nom_%d_%d"%(i,j)).agg({"target":"mean"}),marker="o")
#        plt.title("nom_%d_%d"%(i,j))
#        plt.xticks(rotation=90)
#        plt.show()

In [ ]:
#df.drop(["nom_5","nom_6","nom_7","nom_8","nom_9"],axis=1,inplace=True)

In [ ]:
df.ord_1.unique()

In [ ]:
df["ord_1"] = df.ord_1.replace({"Novice":1,"Contributor":2,"Expert":3,"Master":4,"Grandmaster":5})

In [ ]:
df.ord_2.unique()

In [ ]:
df["ord_2"] = df.ord_2.replace({"Freezing":1,"Cold":2,"Warm":3,"Hot":4,"Boiling Hot":5,"Lava Hot":6})

In [ ]:
df.groupby("ord_1").agg({"target":"mean"})

In [ ]:
df.groupby("ord_2").agg({"target":"mean"})

In [ ]:
df["ord_3"] = df.ord_3.apply(lambda x: ord(x)-64)
df["ord_4"] = df.ord_4.apply(lambda x: ord(x)-64)
df["ord_5_1"] = df.ord_5_1.apply(lambda x: ord(x)-64)
df["ord_5_2"] = df.ord_5_2.apply(lambda x: ord(x)-64)

In [ ]:
df["ord_5_concat"] = df.ord_5_1*100+df.ord_5_2

In [ ]:
df.groupby("ord_5_concat").agg({"target":"mean"})

In [ ]:

plt.plot(np.sort(np.array(df.ord_5_concat.unique())),df.groupby("ord_5_concat").agg({"target":"mean"}),marker=".")
plt.xticks(rotation=90)
plt.show()

In [ ]:
df.drop(["ord_5_1","ord_5_2"],axis=1,inplace=True)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

- `id`: don't have time dependency

In [ ]:
value, bins = pd.qcut(df["id"],100,retbins=True)
bins = ["<"+str(int(i)) for i in bins]
plt.plot(bins[1:],pd.concat([df.target,value],axis=1).groupby("id").agg({"target":"mean"}),marker="o")
plt.xticks(rotation=90)
plt.show()

In [ ]:
def feature_encoding(df):
    df["day"] = np.cos((2*np.pi/6 * (df.day-1)))
    df["ord_5_1"] = df.ord_5.apply(lambda x: x[0])
    df["ord_5_2"] = df.ord_5.apply(lambda x: x[1])

    df = df.drop("ord_5",axis=1)
    df["bin_3"] = df.bin_3.replace({"F":0,"T":1})

    df["bin_4"] = df.bin_4.replace({"N":0,"Y":1})
    for i in ["nom_%d"%i for i in [5,6,7,8,9]]:
        df[i] = df[i].apply(lambda x:int(x,16))
        df[i] = df[i]/df[i].max()
    #df = df.drop(["nom_5","nom_6","nom_7","nom_8","nom_9"],axis=1)
    df["ord_1"] = df.ord_1.replace({"Novice":1,"Contributor":2,"Expert":3,"Master":4,"Grandmaster":5})
    df["ord_2"] = df.ord_2.replace({"Freezing":1,"Cold":2,"Warm":3,"Hot":4,"Boiling Hot":5,"Lava Hot":6})
    df["ord_3"] = df.ord_3.apply(lambda x: ord(x)-64)
    df["ord_4"] = df.ord_4.apply(lambda x: ord(x)-64)
    df["ord_5_1"] = df.ord_5_1.apply(lambda x: ord(x)-64)
    df["ord_5_2"] = df.ord_5_2.apply(lambda x: ord(x)-64)
    df["ord_5_concat"] = df.ord_5_1*100+df.ord_5_2
    df = df.drop(["ord_5_1","ord_5_2"],axis=1)
    for i in df.columns:
        if df[i].dtype == 'object':
            df[i] = df[i].astype("category")
    return df

In [ ]:
test_df = pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")
df = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")

In [ ]:
df = feature_encoding(df)

In [ ]:
X = df.drop(["id","target"],axis=1)
y = df.target

for i in X.columns:
    if X[i].dtype == 'object':
        X[i] = X[i].astype("category")

In [ ]:
test_df = feature_encoding(test_df)

In [ ]:
test_id = test_df.id

In [ ]:
test_df.drop("id",axis=1,inplace=True)

In [ ]:
import category_encoders as ce
smoothing=50

oof = pd.DataFrame([])
for train_idx, oof_idx in StratifiedKFold(
    n_splits=5, random_state=123, shuffle=True).split(
        X, y):
    ce_target_encoder = ce.TargetEncoder(cols = list(X.dtypes[X.dtypes == "category"].index)+["nom_%d"%i for i in [5,6,7,8,9]], smoothing=smoothing)
    ce_target_encoder.fit(X.iloc[train_idx, :], y.iloc[train_idx])
    oof = oof.append(ce_target_encoder.transform(X.iloc[oof_idx, :]), ignore_index=False)

ce_target_encoder = ce.TargetEncoder(cols = list(X.dtypes[X.dtypes == "category"].index)+["nom_%d"%i for i in [5,6,7,8,9]], smoothing=smoothing)
ce_target_encoder.fit(X, y)
X = oof.sort_index() 
test_df = ce_target_encoder.transform(test_df)

In [ ]:
X.columns

In [ ]:
test_df.columns

In [ ]:


kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=11)
pred = []
for train_index, test_index in kf.split(X,y):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    eval_data = lgb.Dataset(X_test, label=y_test, reference= train_data)
    # hyper parameter is imported from other notebook.
    params={
        "learning_rate":0.05,
        'max_depth': 2, 
        'num_leaves': 150,
        'reg_alpha': 0.6, 
        'reg_lambda': 0.6,
        'objective': 'binary',
        "boosting_type": "gbdt",
        "metric": 'auc',
        'random_state': 1}

    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[train_data,eval_data],
        num_boost_round=7000,
        early_stopping_rounds=50,
        verbose_eval=200,
    )
    lgb.plot_importance(gbm,importance_type="gain")
    pred.append(gbm.predict(test_df, num_iteration=gbm.best_iteration))

In [ ]:
pred = np.array(pred).mean(axis=0)
test_id = pd.DataFrame(test_id)
test_id["target"] = pred

sub = pd.read_csv("/kaggle/input/cat-in-the-dat/sample_submission.csv")

test_id.to_csv("submission.csv",index=False)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)
test=scaler.transform(test_df)

In [ ]:
lr_params = {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.123456789, 'max_iter':500}
lin = LogisticRegression(**lr_params)

lin.fit(X, y)
pred_lin = lin.predict_proba(test)[:,1]

In [ ]:
pred = lin.predict_proba(X)[:,1]

In [ ]:
test_id["target"] = pred_lin
test_id.to_csv("submission2.csv",index=False)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,pred)